# Convolutional neural networks (CNNs)

![](../img/depthcol.jpeg)
<pre>         (Image credit: Stanford cs231n <a>http://cs231n.github.io/assets/cnn/depthcol.jpeg)</a></pre>

## Parameters

Each node in convolutional layer is associated with a 3D block (height x width x channel) in the input tensor. Moreover, the convolutional layer itself has multiple output channels. So the layer is parameterized by a 4 dimensional weight tensor, commonly called a convolutional kernel. The output tensor is produced by sliding the kernel across the input image skipping locations according to a pre-defined stride.

In [1]:
#BigDL to express a CNN
#import packages
from __future__ import print_function
import numpy as np
from bigdl.nn.keras.topology import Sequential
from bigdl.nn.keras.layer import *

/home/kunal/anaconda3/envs/deep_learning/lib/python3.6/site-packages/bigdl/util/engine.py:41: UserWarning: Find both SPARK_HOME and pyspark. You may need to check whether they match with each other. SPARK_HOME environment variable is set to: /home/kunal/Downloads/spark-2.4.7-bin-hadoop2.7/, and pyspark is found in: /home/kunal/anaconda3/envs/deep_learning/lib/python3.6/site-packages/pyspark/__init__.py. If they are unmatched, please use one source only to avoid conflict. For example, you can unset SPARK_HOME and use pyspark only.
  warnings.warn(warning_msg)


Prepending /home/kunal/anaconda3/envs/deep_learning/lib/python3.6/site-packages/bigdl/share/conf/spark-bigdl.conf to sys.path


### Convolutional Layer

In BigDL, we use Convolutiona2D() to apply a 2D convolution over an input image composed of several input planes. This function takes a few important parameters: number of convolutional filters(```nbFilter```), number of rows in the convolutional kernal(```nbRow```), number of columns in the convolutional kernal(```nbCol```), string representation of the activation function(```activation```) and the shape of the input layer(```input_shape```).

In [2]:
#Define Layers
model = Sequential()
model.add(Convolution2D(20, 3, 3, activation="relu", input_shape=(1, 28, 28)))
input = np.random.random([2, 1, 28, 28])
output = model.forward(input)

/home/kunal/anaconda3/envs/deep_learning/lib/python3.6/site-packages/bigdl/nn/keras/topology.py:213: UserWarning: bigdl.nn.keras is deprecated in 0.11. Recommend to use Analytics Zoo's Keras API.
  warnings.warn("bigdl.nn.keras is deprecated in 0.11. "


creating: createKerasSequential
creating: createKerasConvolution2D


**Note** 

The number of examples (64) remains unchanged. The number of channels (also called filters) has increased to 20. And because the (3,3) kernel can only be applied in 26 different heights and widths (without the kernel busting over the image border), our output is 26,26. There are some weird padding tricks we can use when we want the input and output to have the same height and width dimensions, but we won’t get into that now.

### Average pooling

In [3]:
#Average pooling, default value for downscaling the weights vertically and horizontally is (2, 2)
model = Sequential()
model.add(MaxPooling2D(input_shape = (1, 28, 28)))
input = np.random.random([2, 1, 28, 28])
output = model.forward(input)

creating: createKerasSequential
creating: createKerasMaxPooling2D


### Flatten Layer

In [4]:
#It flattens the input without affecting the batch size
model = Sequential()
model.add(Flatten(input_shape=(2, 2, 3)))
input = np.random.random([2, 2, 2, 3])
output = model.forward(input)
print(input)
output

creating: createKerasSequential
creating: createKerasFlatten
[[[[0.18321077 0.6766386  0.71398925]
   [0.09295621 0.82980218 0.76773918]]

  [[0.5726227  0.8689477  0.49284268]
   [0.53057877 0.52126079 0.14199241]]]


 [[[0.54276957 0.53055823 0.51371165]
   [0.77832433 0.87990441 0.49560211]]

  [[0.94963403 0.47248744 0.65917649]
   [0.49838467 0.60026824 0.06189344]]]]


array([[0.18321078, 0.6766386 , 0.71398926, 0.09295621, 0.82980216,
        0.7677392 , 0.5726227 , 0.8689477 , 0.49284267, 0.5305788 ,
        0.5212608 , 0.1419924 ],
       [0.54276955, 0.5305582 , 0.51371163, 0.7783243 , 0.8799044 ,
        0.4956021 , 0.949634  , 0.47248745, 0.65917647, 0.49838468,
        0.60026824, 0.06189344]], dtype=float32)

### MNIST data

In [5]:
#Getting MINIST Data
from bigdl.dataset import mnist
from bigdl.util.common import *

mnist_path = "./data/mnist"
(X_train, Y_train), (X_test, Y_test) = mnist.load_data(mnist_path)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz
(60000, 28, 28, 1)
(10000, 28, 28, 1)
(60000,)
(10000,)


In [6]:
#Convolutional neural network model
num_fc = 512
num_outputs = 10
model = Sequential()
model.add(Reshape((1, 28, 28), input_shape=(28, 28, 1)))
model.add(Convolution2D(20, 3, 3, activation="relu", input_shape=(1, 28, 28)))
model.add(MaxPooling2D())
model.add(Convolution2D(50, 3, 3, activation="relu", name="conv2_5x5"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(num_fc, activation="relu", name="fc1"))
model.add(Dense(num_outputs, activation="softmax", name="fc2"))
#Print Model input and output shape
print(model.get_input_shape())
print(model.get_output_shape())

creating: createKerasSequential
creating: createKerasReshape
creating: createKerasConvolution2D
creating: createKerasMaxPooling2D
creating: createKerasConvolution2D
creating: createKerasMaxPooling2D
creating: createKerasFlatten
creating: createKerasDense
creating: createKerasDense
(None, 28, 28, 1)
(None, 10)


In [7]:
#Training configure
from bigdl.nn.criterion import *

model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='sgd',
                metrics=['accuracy'])

creating: createDefault
creating: createSGD
creating: createClassNLLCriterion
creating: createTop1Accuracy


In [ ]:
#Execute training
model.fit(X_train, Y_train, batch_size=8, nb_epoch=1,validation_data=(X_test, Y_test))